## _Import_

In [1]:
import tensorflow as tf
import numpy as np
import random
import time
import os

from tensorflow.keras import layers

In [2]:
tf.__version__

'2.2.0'

## _Dummy Data_

In [ ]:
data = [np.ones([1,4, 4,1], dtype = int) for x in range(100)] 

In [ ]:
data[5].shape

In [ ]:
data[5]

## _The Generator_

In [30]:
def generator_model(n):
    model = tf.keras.Sequential()
    model.add(layers.Dense(n*n*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((n, n, 256)))
    assert model.output_shape == (None, n, n, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, n, n, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 2*n, 2*n, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 2*n, 2*n, 1)
    return model

In [31]:
def discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[4, 4, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
generator = generator_model(2)

In [ ]:
generator.summary()

In [ ]:
discriminator = discriminator_model()

In [ ]:
discriminator.summary()

### _Generator Test_

In [ ]:
noise = tf.random.normal([1, 100])
generated_matrix = generator(noise, training=False)

In [ ]:
generated_matrix

### _Discriminator Test_

In [ ]:
test = tf.reshape(data[0], [1,4,4,1])
test

In [ ]:
discriminator(test)

In [ ]:
discriminator(generated_matrix)

In [ ]:
decision = discriminator(generated_matrix)
print (decision)

## _Loss and Optimizer_

In [32]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def D_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
  
def G_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [33]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [34]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## _Training Functions_

In [35]:
noise_dim = 100
num_of_generated_examples = 16

seed = tf.random.normal([num_of_generated_examples, noise_dim])

In [36]:
generator = generator_model(2)
discriminator = discriminator_model()

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

#@tf.function
def train_step(adj_matrix):
    noise = tf.random.normal([100, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_matrix = generator(noise, training=True)
        
        real_output = discriminator(adj_matrix, training=True)
        fake_output = discriminator(generated_matrix, training=True)

        gen_loss = G_loss(fake_output)
        disc_loss = D_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

## _Train_

In [37]:
def train_GAN(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        
        for batch in dataset:
            train_step(batch)

        # Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [ ]:
train_GAN(data, 500)

In [ ]:
noise = tf.random.normal([1, 100])
gen_2 = generator(noise, training=False)

In [ ]:
gen_2

## _Saving Model_

In [ ]:
generator.save('Model/')

In [ ]:
enigma = tf.keras.models.load_model('Model/', compile=False)

In [ ]:
enigma

In [ ]:
enigma.summary()

In [ ]:
noise = tf.random.normal([1, 100])

In [ ]:
noise

In [ ]:
enigma(noise, training=False)

## _Adj Matrix_

In [11]:
def random_adjacency_matrix(n):
    """
    creates an nxn symmetric adjacency matrix with 0s along the diagonal
    used to represent an undirected graph with n nodes
    :param n: dimension
    :return: nxn numpy array
    """
    matrix = [[random.randint(0, 1) for i in range(n)] for j in range(n)]
    # No vertex connects to itself
    for i in range(n):
        matrix[i][i] = 0
    # If i is connected to j, j is connected to i
    for i in range(n):
        for j in range(n):
            matrix[j][i] = matrix[i][j]
    return np.array(matrix)

In [12]:
random_adjacency_matrix(4)

array([[0, 1, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [0, 0, 1, 0]])

In [13]:
data = [random_adjacency_matrix(4).reshape([1,4,4,1]) for x in range(500)] 

In [ ]:
test = data[0]

In [ ]:
test

In [ ]:
test.reshape([4,4])

In [ ]:
data[0]

## _Train Enigma 2 with better Dummy Data_

In [38]:
data_2 = [random_adjacency_matrix(4).reshape([1,4,4,1]) for x in range(100)] 

In [39]:
train_GAN(data_2, 100)

Time for epoch 1 is 2.7683157920837402 sec
Time for epoch 2 is 2.6759774684906006 sec
Time for epoch 3 is 2.647869110107422 sec
Time for epoch 4 is 2.832427978515625 sec
Time for epoch 5 is 2.7549357414245605 sec
Time for epoch 6 is 2.7482006549835205 sec
Time for epoch 7 is 2.8244383335113525 sec
Time for epoch 8 is 2.900914192199707 sec
Time for epoch 9 is 2.6654133796691895 sec
Time for epoch 10 is 2.6945090293884277 sec
Time for epoch 11 is 2.712099552154541 sec
Time for epoch 12 is 2.6996915340423584 sec
Time for epoch 13 is 2.6906418800354004 sec
Time for epoch 14 is 2.680401563644409 sec
Time for epoch 15 is 2.7673134803771973 sec
Time for epoch 16 is 2.7330236434936523 sec
Time for epoch 17 is 2.700709342956543 sec
Time for epoch 18 is 2.707001209259033 sec
Time for epoch 19 is 2.799954414367676 sec
Time for epoch 20 is 2.703674554824829 sec
Time for epoch 21 is 2.659414052963257 sec
Time for epoch 22 is 2.6714985370635986 sec
Time for epoch 23 is 2.673480272293091 sec
Time for

In [40]:
generator.save('Model/')

INFO:tensorflow:Assets written to: Model/assets


In [41]:
enigma = tf.keras.models.load_model('Model/', compile=False)

In [42]:
enigma.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1024)              102400    
_________________________________________________________________
batch_normalization_6 (Batch (None, 1024)              4096      
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 2, 2, 256)         0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 2, 2, 128)         819200    
_________________________________________________________________
batch_normalization_7 (Batch (None, 2, 2, 128)         512       
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 2, 2, 128)        

In [43]:
noise = tf.random.normal([1, 100])
cat = enigma(noise, training=False)

In [44]:
cat

<tf.Tensor: shape=(1, 4, 4, 1), dtype=float32, numpy=
array([[[[-0.00650752],
         [ 0.24790123],
         [ 0.31918916],
         [ 0.04215904]],

        [[ 0.33064163],
         [ 0.0209253 ],
         [ 0.82438385],
         [ 0.2521356 ]],

        [[ 0.15727054],
         [ 0.84887254],
         [ 0.04201905],
         [ 0.921328  ]],

        [[-0.0090999 ],
         [ 0.24500865],
         [ 0.907346  ],
         [-0.01367382]]]], dtype=float32)>

In [48]:
test = tf.reshape(cat, [4,4])

In [49]:
test

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[-0.00650752,  0.24790123,  0.31918916,  0.04215904],
       [ 0.33064163,  0.0209253 ,  0.82438385,  0.2521356 ],
       [ 0.15727054,  0.84887254,  0.04201905,  0.921328  ],
       [-0.0090999 ,  0.24500865,  0.907346  , -0.01367382]],
      dtype=float32)>

In [58]:
bo = tf.math.greater(test, 0.5)

In [62]:
tf.cast(bo, tf.int32)

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[0, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 1],
       [0, 0, 1, 0]], dtype=int32)>